7.1 통계적 가설 검정
- 가설(hypothesis) : 통계치들은 다양한 가정하 특정 분포에 대한 확률변수의 관측치; 그런 과정들의 타당함

- 귀무가설(H0, null hypothesis) : 기본적인 가설
- 대립가설(H1, alternative hypothesis) : 비교하고 싶은 가설

7.2 e.g. 동전 던지기
 - 동전이 앞면이 나올 확률 p
 - 귀무가설 : p = 0.5 
 - 대립가설 : p != 0.5

In [1]:
import math, random

In [2]:
def normal_approximation_to_binormal(n,p):
    # Binomal(n,p)에 해당하는 mu(평균)와 sigma(표준편차) 계산
    mu = p*n
    sigma = math.sqrt(p * (1-p) * n)
    
    return mu, sigma

In [3]:
mu, sigma = normal_approximation_to_binormal(1000, 0.5)

mu, sigma

(500.0, 15.811388300841896)

확률변수가 정규분포를 따른다는 전제하, normal_cdf를 사용하여 실제 동전 던지기로부터 얻은 값이 구안 안(혹은 밖)에 존재할 확률 계산

In [4]:
def normal_cdf(x, mu=0,sigma=1):
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2

# 누적분포함수는 확률변수가 특정 값보다 작을 확률을 나타냄
normal_probability_below = normal_cdf

In [5]:
# 만약 확률변수가 특정 값보다 작지 않다면, 특정 값보다 크다는 것을 의미
def normal_probability_above(lo, mu=0, sigam=1):
    return 1 - normal_cdf(lo, mu, sigma)

# 만약 확률변수가 hi보다 작고 lo보다 작지 않다면, 확률변수는 hi와 lo 사이에 존재
def normal_probability_beween(lo, hi, mu=0, sigma=1):
    return normal_cdf(hi,mu,sigma) - normal_cdf(lo,mu,sigma)

# 만약 확률변수가 범위 밖이라면, 범위 안에 존재하지 x 의미
def normal_probability_outside(lo, hi, mu=0, sigma=1):
    return 1 - normal_probability_between(lo, hi, mu, sigma)

반대로, 확률이 주어지고, 평균을 중심으로 하는 (대칭적인)구간을 구할 수 있음.
e.g. 분포의 60%를 차지하는 평균 중심의 구간을 구하고 싶다면 꼬리 부분이 각각 분포의 20%를 차지하는 지점을 구하면 됨.

In [6]:
def inverse_normal_cdf(p, mu=0, sigma=1, tolerance=0.00001): #tolerance = 공차 #모든 수가 같은 비중을 둔다는 의미.
    """find approximate inverse using binary search"""

    # 표준정규분포가 아니면 표준정규분포로 변환()
    if mu != 0 or sigma != 1:
        return mu + sigma * inverse_normal_cdf(p, tolerance=tolerance)
    
    #수평축 값의 범위가 -10 ~ 10
    low_z, low_p = -10.0, 0            # normal_cdf(-10) is (very close to) 0
    hi_z,  hi_p  =  10.0, 1            # normal_cdf(10)  is (very close to) 1

    #표준정규분포의 구간을 반복적으로 이등분하되, 원하는 확률값에 가까워질 때까지 계속 나눈다(hi_z - low_z =< tolerance일 때까지)
    while hi_z - low_z > tolerance:
        mid_z = (low_z + hi_z) / 2     # consider the midpoint
        mid_p = normal_cdf(mid_z)      # and the cdf's value there
        if mid_p < p:
            # midpoint is still too low, search above it
            low_z, low_p = mid_z, mid_p
        elif mid_p > p:
            # midpoint is still too high, search below it
            hi_z, hi_p = mid_z, mid_p
        else:
            break

    return mid_z

In [7]:
def normal_upper_bound(probability, mu=0, sigma=1):
    # P(Z <= z)  = 0 probability인 z값 반환
    return inverse_normal_cdf(probability, mu, sigma)
    
def normal_lower_bound(probability, mu=0, sigma=1):
    # P(Z >= z) = 0 probability인 z 값 반환
    return inverse_normal_cdf(1 - probability, mu, sigma)

def normal_two_sided_bounds(probability, mu=0, sigma=1):
    # 입력한 probability값을 포함하고, 평균을 중심으로 대칭적인 구간 반환
    tail_probability = (1 - probability) / 2

    # 구간의 상한은 tail_probability 값 이상의 확률 값을 갖고 있음.
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)

    # 구간의 하한은 tail_probability 값 이하의 확률 값을 갖고 있음.
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)

    return lower_bound, upper_bound

In [8]:
# 동전 1000번 던지기 (n = 1000)
# 가설이 맞다면 X는 대략 평균 : 50, 표준편차 : 15.8인 정규분포
mu_0, sigma_0 = normal_approximation_to_binormal(1000, 0.5)
normal_two_sided_bounds(0.95, mu_0, sigma_0)

(469.01026640487555, 530.9897335951244)

In [9]:
# p가 0.5라 가정시, 유의수준 5%구간
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)
print(lo, hi)

# p가 0.55인 경우 실제 평균과 표준편차
mu_1, sigma_1 = normal_approximation_to_binormal(1000,0.55)

# 제2종 오류란 귀무가설(H0)을 기각하지 못한다는 의미
# 즉, X가 주어진 구간 안에 존재할 경우를 의미
type_2_probability = normal_probability_beween(lo, hi, mu_1, sigma_1)
power = 1 - type_2_probability
print(power)

469.01026640487555 530.9897335951244
0.8865480012953671


In [10]:
# p < 0.5, 즉 동전이 앞면에 편향되지 않은 경우를 귀무가설로 정하면 X가 50보다 크면 귀무가설을 기각하고,
# 50보다 작다면 기각하지 않는 단측검정이 필요.
# 유의수준이 5%인 가설검정을 위해서 normal_probability_below를 사용하여 분포의 95%가
# 이하인 경계값을 찾을 수 있다.

hi = normal_upper_bound(0.95, mu_0, sigma_0)
print(hi) # 526 (<531, 분포 상위 부분에 더 높은 확률을 주기 위해)

type_2_probability = normal_probability_below(hi, mu_1, sigma_1)

power = 1 - type_2_probability
print(power) # 0.936

# 이 가설검정은 더 이상 X가 469보다 작을 때 H0을 기각하는게 아니라 (H1이 참이라면 이는 거의 발생하지 x)
# X가 526에서 531사이 일 때, H0을 기각하기 때문에(H1이 참이라면 이는 가끔 발생) 전보다 검정력이 더 좋아졌다 볼 수 있음.

526.0073585242053
0.9363794803307173


7.3 p-value
 - 가설을 바라보는 또 다른 관점
 - 어떤 확률값을 기준으로 구간을 선택하는 대신, H0가 참이라는 가정하 실제로 관측된 값보다 더 극단적인 값이 나올 확률을 구하는 것.

In [11]:
# 동전이 공평한지 확인하기 위한 양측검장
def two_sided_p_value(x, mu=0, sigma=1):
    if x >= mu:
        # 만약 x가 평균보다 크면, x보다 큰 부분이 꼬리
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        # 만약 x가 평균보다 작다면, x보다 작은 부분이 꼬리
        return 2 * normal_probability_below(x, mu, sigma)

In [12]:
two_sided_p_value(529.5, mu_0, sigma_0)

0.06207721579598857

530이 아니라 529.5를 사용한 이유? 연속수정(continuity correction) 때문
즉, 동전의 앞면이 530번 나올 확률은 normal_probability_between(530,531,mu_0,sigma_0)보다 normal_probability_between(529.5,530.5,mu_0,sigma_0)을 사용시 더 정확.

동전의 앞면이 적어도 530번 나올 확률 또한 normal_probability_above(529.5,mu_0,sigma_0)을 사용하는 것이 더 정확.

In [13]:
extreme_value_count = 0
for _ in range(100000):
    num_heads = sum(1 if random.random() < 0.5 else 0    # 앞면 카운트
                    for _ in range(1000))                # 1000번 던짐
    if num_heads >= 530 or num_heads <= 470:             # 극한값이
        extreme_value_count += 1                         # 얼마나 나오는 지

print(extreme_value_count / 100000)

0.06212


계산된 p-value가 5%보다 크기 떄문에 귀무가설 기각하지 x.

만약 동전의 앞면이 532번 나왔다면 p-value는 5%보다 작을 것이고, 이 경우 귀무가설 기각

In [14]:
two_sided_p_value(531.5, mu_0, sigma)

0.046345287837786575

In [15]:
# 같은 방식으로
upper_p_value = normal_probability_above
lower_p_value = normal_probability_below

In [16]:
# 동전의 앞면이 525번 나왔다면 단측검정을 위한 p-value는 다음과 같이 귀무가설 기각x
upper_p_value(524.5, mu_0, sigma_0)

0.06062885772582083

In [17]:
# 만약 동전의 앞면이 527번 나왔다면, p-value는 다음과 같이 귀무가설 기각
upper_p_value(526.5, mu_0, sigma_0)

0.04686839508859242

# nomal_probability로 p-value 계산전, 주어진 데이터가 정규분포인지 확인은 필수!

# e.g. 데이터를 시각화하여 정규분포를 따르는 지 확인 한다던가

7.4 신뢰구간(confidence interval)
 - (앞면이 나올 확률인 p인)동전 던지기를 예시로 했을 때, p는 앞면이 나올 미지의 분포를 나타내는 파라미터
 - 사건에 대한 분포를 모른다면 -> 관측 값에 대한 신뢰구간을 사용하여 가설 검정 가능

e.g. 공평치 않는 동전에 대한 확률에서 앞면1, 뒷면0

베르누이 확률 변수의 평군을 이용하여 추정 가능.

만약 동전을 1000번 던져서 앞면이 525번 나오면, p = 0.525로 추정 가능

math.sqrt(p * (1-p)/1000)

In [20]:
p_hat = 525/1000
mu = p_hat
sigma = math.sqrt(p_hat * (1-p_hat)/1000)

lower, upper = normal_two_sided_bounds(0.95, mu, sigma)

lower < 0.5  < upper

True

In [21]:
# 정규분포의 근사를 사용시, 진짜 p가 다음 구간안에 포함되어 있을거라는 '95%확신'가능
normal_two_sided_bounds(0.95, mu, sigma)
# 위 의 경우, 0.5는 신뢰구간 안에 있기 떄문에, 동전은 공평하지 않다고 결론을 내릴 수 x

(0.4940490278129096, 0.5559509721870904)

In [22]:
# 반대로, 앞면이 540번이라면
p_hat = 540 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)
normal_two_sided_bounds(0.95, mu, sigma)

lower, upper = normal_two_sided_bounds(0.95, mu, sigma)

lower < 0.5  < upper

# '공평한 동전' 에 대한 확률은 계산된 신뢰구간 밖에 존재.
# 공평한 동전이라는 가설은, 가설이 참이라면 모든 경우의 95%에 대해 참인 가설검정을 통과하지x

False

7.5 p-value 해킹
귀무가설을 잘못 기가하는 경우가 5%인 가설검정은 정의에서 알 수 있듯,

모든 경우의 5%에서 귀무가설을 잘못 기각

In [25]:
def run_experiment():
    # 동전 1000번 던져, True = 앞면, False = 뒷면
    return [random.random() < 0.5 for _ in range(1000)]

def reject_fairness(experiment):
    # 유의수준 5%
    num_heads = len([flip for flip in experiment if flip])
    return num_heads < 469 or num_heads > 531

random.seed(0)
experiments = [run_experiment() for _ in range(1000)]
num_rejections = len([experiment
                     for experiment in experiments
                     if reject_fairness(experiment)])
print(num_rejections)

46


'의미 있는' 결과를 찾으려고 '노력'한다면 보통 의미 있는 결과를 찾을 수 있다.
p-value의 관점에서 추론을 하면 'p-value 해킹'이 발생할 수 있다.

중요 포인트 세가지
    1. 가설은 데이터를 보기 전에 세운다.
    2. 데이터 처리시 세운 가설은 잠시 잊는다.
    3. p-value가 전부가 아니다. (베이지안 추론 사용 가능)

7.6 예시: A/B test

데이터에서 주요 임무 중 하나는 사용자의 경험 최적화. (e.g. 사용자들에게 광고를 더 자주 클릭하게 하기)

광고 a를 본 1000명 중 990명 클릭

광고 b를 본 1000명 중 10명 광고 클릭

NA명의 사용자가 aA를 보았고, 그중 nA명이 광고를 클릭했다고 가정.

각 사용자가 광고 A를 클릭할 확률 : pA (만약 NA가 큰 숫자라 가정)

nA/NA는 평균이 pA, 표준편차 σA = math.sqrt(PA(1-PA)/NA)

In [26]:
def estimated_parameters(N, n):
    p = n / N
    sigma = math.sqrt(p * (1 - p) / N)
    return p, sigma

# 만약 두 정규분포가 독립이라면 (각각의 베르누이 시행은 독립적인 시행이기에 적절한 가정),
# 두 정규분포의 차이 또한 평균이 pb- pa 이고 표준편차가 math.sqrt(σ^2A + σ^2B)인 정규분포 

In [28]:
# pA와 pB가 같다는 (즉, pA-pB=0) 귀무가설은 다음과 같은 통계치로 검정
def a_b_test_statistic(N_A, n_A, N_B, n_B):
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    return (p_B - p_A) / math.sqrt(sigma_A ** 2 + sigma_B ** 2)

# 1000명중 200명 A광고 클릭, 180명이 B광고 클릭
z = a_b_test_statistic(1000,200,1000,180)
z

-1.1403464899034472

In [30]:
# 값이 크기 때문에, 두 분포가 다르다고 결론을 내릴 수 x
two_sided_p_value(z)
# 두 광고 효과의 평균이 같을 확률이 높음(p > 0.05) = 광고 효과가 같다.

0.254141976542236

In [31]:
# 만약 150명이 B광고를 클릭시,
z = a_b_test_statistic(1000,200,1000,150)
two_sided_p_value(z)
# 즉, A B 두 광고가 동일하게 효과적이라면 이렇게 큰 차이가 발생할 확률은 불과 0.003
# 두 광고 효과의 평균이 같을 확률이 매우 낮다 (p < 0.05) = 광고효과가 다르다.

0.003189699706216853

7.7 베이지안 추론

    -위의 언급된 방법들은 통계적 검정을 확률적으로 설명하는 것(e.g. 귀무가설이 사실이라면 이렇게 극단적인 통계치가 발생할 확률은 3%)

    -베이지안 추론은 알려지지 않은 파라미터를 확률변수로 보는 방법

    -통계적 검정에 대해 확률적으로 결론을 내는 대신에 파라미터에 대해 확률적으로 결론을 낸다.

In [32]:
def B(alpha, beta):
    # 모든 확률 값의 합이 1이 되도록 해주는 정규화 값
    return math.gamma(alpha) * math.gamma(beta) / math.gamma(alpha + beta)

def beta_pdf(x, alpha, beta):
    if x < 0 or x > 1:          # [0,1] 구간 밖에서느 밀도가 x  
        return 0        
    return x ** (alpha - 1) * (1 - x) ** (beta - 1) / B(alpha, beta)

Reference :

밑바닥부터 시작하는 데이터과학 7장,
https://github.com/newhiwoong/DATA_SCIENCE_FROM_SCRATCH